# 데이터셋 불러오기

In [1]:
#!sudo apt-get install -y fonts-nanum
#!sudo fc-cache -fv

#!rm ~/.cache/matplotlib -rf
#!pip install catboost

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
  # 1순위컬럼 결측치 그대로 이상치제거
fire_df = pd.read_csv("/content/drive/MyDrive/화재피해금액예측/1순위_2순위컬럼합_재정비.csv" , index_col='연번',encoding = 'CP949')

In [6]:
fire_df.drop('펌프물탱크', axis=1, inplace=True)
fire_df.drop('건물구조동', axis=1, inplace=True)
# fire_df.drop('소방서거리(㎞)', axis=1, inplace=True)
# fire_df.drop('안전센터거리', axis=1, inplace=True)
# fire_df.drop('지역대거리', axis=1, inplace=True)
# fire_df.drop('피난기구',axis=1,inplace=True)

In [7]:
plt.rc('font', family='NanumBarunGothic') 

# 데이터 전처리

In [8]:
fire_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11259 entries, 1 to 44175
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   재산피해소계      11259 non-null  int64  
 1   화재피해경감액     11259 non-null  int64  
 2   시도          11259 non-null  object 
 3   시군구         11259 non-null  object 
 4   건물상태        11259 non-null  object 
 5   화재보험의무가입대상  11259 non-null  object 
 6   건물구조식       11246 non-null  object 
 7   건물구조조       11259 non-null  object 
 8   건물구조즙       11258 non-null  object 
 9   연면적         11258 non-null  float64
 10  바닥면적        11257 non-null  float64
 11  장소중분류       11259 non-null  object 
 12  장소소분류       11259 non-null  object 
 13  부속용도소분류     11244 non-null  object 
 14  자동화재감지기     1308 non-null   object 
 15  건물층수(지상)    11258 non-null  float64
 16  건물층수(지하)    9797 non-null   float64
 17  소방서거리(㎞)    11259 non-null  float64
 18  안전센터거리      11259 non-null  float64
 19  지역대거리       11246 non-nul

In [9]:
fire_df['재산'] = fire_df['재산피해소계']+fire_df['화재피해경감액']
fire_df.drop('화재피해경감액', axis=1, inplace=True)

정답값 범주화

In [10]:
# y값 범주화
bins_1 = [-1,2327,4653,52124,100000]
labels_1= [1,2,3,4]
#labels_1= ['1구간','2구간','3구간','4구간']
fire_df['재산피해소계'] = pd.cut(fire_df['재산피해소계'], bins=bins_1, labels=labels_1)
fire_df = fire_df.astype({'재산피해소계':'int'})

In [11]:
# # 이상치 0으로 변환
# for i in fire_df.index :
#     if fire_df[ '소방서거리(㎞)' ][i]  > 90 :
#         fire_df[ '소방서거리(㎞)' ][i] = 0

# 소방서거리(㎞) 범주화
bins_1 = [-1, 11, 21, 31, 41, 51, 61, 71, 81, 91, 100]
labels_1= [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
fire_df['소방서거리(㎞)'] = pd.cut(fire_df['소방서거리(㎞)'], bins=bins_1, labels=labels_1)
fire_df = fire_df.astype({'소방서거리(㎞)':'int'})

In [12]:
# 이상치 0으로 변환
count = 0
for i in fire_df.index :
    if fire_df[ '안전센터거리' ][i]  > 90 :
        fire_df[ '안전센터거리' ][i] = 0
        count+=1
print(count)

# 안전센터거리 범주화
bins_1 = [-1, 3.52, 4.4, 6.16, 11.44, 14.96, 22, 28.16, 100]
labels_1= [1, 2, 3, 4, 5, 6, 7, 8]
fire_df['안전센터거리'] = pd.cut(fire_df['안전센터거리'], bins=bins_1, labels=labels_1)
fire_df = fire_df.astype({'안전센터거리':'int'})

3


In [13]:
# # 지역대거리 범주화
# fire_df['지역대거리'] = fire_df['지역대거리'].fillna(0)
# bins_1 = [-1, 0.43, 7.99, 100]
# labels_1= [1, 2, 3]
# fire_df['지역대거리'] = pd.cut(fire_df['지역대거리'], bins=bins_1, labels=labels_1)
# fire_df = fire_df.astype({'지역대거리':'int'})

# # 값이 있으면 1, 값이 없으면 0
# for i in fire_df.index :
#     if fire_df[ '지역대거리' ][i] > 0 :
#         fire_df[ '지역대거리' ][i] = 1     

## 추가컬럼코드

In [14]:
fire_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11259 entries, 1 to 44175
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   재산피해소계      11259 non-null  int64  
 1   시도          11259 non-null  object 
 2   시군구         11259 non-null  object 
 3   건물상태        11259 non-null  object 
 4   화재보험의무가입대상  11259 non-null  object 
 5   건물구조식       11246 non-null  object 
 6   건물구조조       11259 non-null  object 
 7   건물구조즙       11258 non-null  object 
 8   연면적         11258 non-null  float64
 9   바닥면적        11257 non-null  float64
 10  장소중분류       11259 non-null  object 
 11  장소소분류       11259 non-null  object 
 12  부속용도소분류     11244 non-null  object 
 13  자동화재감지기     1308 non-null   object 
 14  건물층수(지상)    11258 non-null  float64
 15  건물층수(지하)    9797 non-null   float64
 16  소방서거리(㎞)    11259 non-null  int64  
 17  안전센터거리      11259 non-null  int64  
 18  지역대거리       11246 non-null  float64
 19  소하기구(종류)    2787 non-null

In [15]:
fire_df.corr()

,재산피해소계,연면적,바닥면적,건물층수(지상),건물층수(지하),소방서거리(㎞),안전센터거리,지역대거리,재산
재산피해소계,1.000000,-0.006038,-0.009624,-0.161572,-0.158804,0.214430,0.262399,0.151915,0.023621
연면적,-0.006038,1.000000,0.060034,0.035965,0.006981,-0.009089,-0.010543,-0.005662,0.087716
바닥면적,-0.009624,0.060034,1.000000,0.048621,0.026043,-0.013602,-0.009354,-0.005392,0.120874
건물층수(지상),-0.161572,0.035965,0.048621,1.000000,0.530610,-0.230609,-0.295887,-0.169223,0.069771
건물층수(지하),-0.158804,0.006981,0.026043,0.530610,1.000000,-0.246302,-0.303908,-0.170734,0.025095
소방서거리(㎞),0.214430,-0.009089,-0.013602,-0.230609,-0.246302,1.000000,0.608646,0.429722,-0.012721
안전센터거리,0.262399,-0.010543,-0.009354,-0.295887,-0.303908,0.608646,1.000000,0.487812,-0.016777
지역대거리,0.151915,-0.005662,-0.005392,-0.169223,-0.170734,0.429722,0.487812,1.000000,-0.009567
재산,0.023621,0.087716,0.120874,0.069771,0.025095,-0.012721,-0.016777,-0.009567,1.000000


결측치 정리(0으로 채우기)

In [16]:
fire_df = fire_df.fillna(0)

# train,test 분리하기

In [17]:
X = fire_df.loc[:, '시도':]
y = fire_df["재산피해소계"]

In [18]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
#                                                    random_state = 3)


# 원 핫 인코딩

In [19]:
# X_train = pd.get_dummies(X_train)
# X_test = pd.get_dummies(X_test)
X = pd.get_dummies(X)
y = pd.get_dummies(y)

In [20]:
# a = set(X_train.columns) - set(X_test.columns)
# a = list(a)
# for i in range(len(a)):
#     X_test[a[i]]= 0
#     tmp = X_train[a[i]]
#     X_train.drop(a[i],axis =1, inplace=True)
#     X_train = pd.concat([X_train,tmp],axis=1)

In [21]:
# b = set(X_test.columns) - set(X_train.columns)
# b = list(b)
# for i in range(len(b)):
#     X_train[b[i]]= 0
#     tmp = X_test[b[i]]
#     X_test.drop(b[i],axis =1, inplace=True)
#     X_test = pd.concat([X_test,tmp],axis=1)

# 스케일링

In [22]:
scaler = StandardScaler()
# scaler.fit(X_train)
# scaler.fit(X_test)
scaler.fit(X)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [23]:
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
X = scaler.transform(X)

In [24]:
# print(X_train.shape)
# print(X_test.shape)
print(X.shape)

(11259, 390)


In [25]:
# print(y_train.shape)
# print(y_test.shape)
print(y.shape)

(11259, 4)


# 모델 생성하기

In [26]:
# lgb_model = LGBMClassifier(n_estimators = 300, learning_rate=0.01, random_state=123) #기존 lr : 0.1

In [27]:
# lgb_model.fit(transform_X_train, y_train)

## 딥러닝 모델 생성

In [28]:
# tensorflow와 tf.keras를 임포트합니다
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [29]:
# 1. 신경망 구조 설계
baseline_model = Sequential()    # 신경망 모델의 뼈대를 구축
# 입력층 + 중간층
baseline_model.add(Dense(16, input_dim=390,  activation="relu"))
# 중간층
baseline_model.add(Dense(32, activation="relu"))
baseline_model.add(Dense(8, activation="relu"))
# 출력층
# 정답 데이터의 원핫인코딩 된 레이블 수, 다중분류 활성화함수 : softmax
baseline_model.add(Dense(4, activation="softmax"))
baseline_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                6256      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 36        
Total params: 7,100
Trainable params: 7,100
Non-trainable params: 0
_________________________________________________________________


In [30]:
# 1. 신경망 구조 설계
l2_model = Sequential()    # 신경망 모델의 뼈대를 구축
# 입력층 + 중간층
l2_model.add(Dense(16, input_dim=390, kernel_regularizer=keras.regularizers.l2(0.001),
                   activation="relu"))
# 중간층
l2_model.add(Dense(32, kernel_regularizer=keras.regularizers.l2(0.001), activation="relu"))
l2_model.add(Dense(8, kernel_regularizer=keras.regularizers.l2(0.001), activation="relu"))
# 출력층
# 정답 데이터의 원핫인코딩 된 레이블 수, 다중분류 활성화함수 : softmax
l2_model.add(Dense(4, activation="softmax"))
l2_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 16)                6256      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 36        
Total params: 7,100
Trainable params: 7,100
Non-trainable params: 0
_________________________________________________________________


In [31]:
# 2. 학습평가/방법 설정
# 다중분류 비용(손실)함수 : categorical_crossentropy
baseline_model.compile(loss="categorical_crossentropy",
              optimizer="Adam",
              metrics=["acc"]
              )

In [32]:
# 2. 학습평가/방법 설정
# 다중분류 비용(손실)함수 : categorical_crossentropy
l2_model.compile(loss="categorical_crossentropy",
              optimizer="Adam",
              metrics=["acc"]
              )

In [33]:
# print(X_train.shape)
# print(y_train.shape)

In [34]:
# 3. 학습
# model.fit(X_train, y_train, epochs=300)
baseline_history = baseline_model.fit(X, y, epochs=300)

Epoch 1/300
352/352 [==============================] - 1s 2ms/step - loss: 0.8484 - acc: 0.6906
Epoch 2/300
352/352 [==============================] - 1s 2ms/step - loss: 0.7282 - acc: 0.7193
Epoch 3/300
352/352 [==============================] - 1s 2ms/step - loss: 0.7064 - acc: 0.7275
Epoch 4/300
352/352 [==============================] - 1s 2ms/step - loss: 0.6916 - acc: 0.7320
Epoch 5/300
352/352 [==============================] - 1s 2ms/step - loss: 0.6802 - acc: 0.7367
Epoch 6/300
352/352 [==============================] - 1s 2ms/step - loss: 0.6680 - acc: 0.7401
Epoch 7/300
352/352 [==============================] - 1s 2ms/step - loss: 0.6556 - acc: 0.7471
Epoch 8/300
352/352 [==============================] - 1s 2ms/step - loss: 0.6452 - acc: 0.7515
Epoch 9/300
352/352 [==============================] - 1s 2ms/step - loss: 0.6357 - acc: 0.7547
Epoch 10/300
352/352 [==============================] - 1s 2ms/step - loss: 0.6245 - acc: 0.7588
Epoch 11/300
352/352 [=================

In [35]:
# 3. 학습
# model.fit(X_train, y_train, epochs=300)
l2_model_history = l2_model.fit(X, y, epochs=300)

Epoch 1/300
352/352 [==============================] - 1s 2ms/step - loss: 0.8792 - acc: 0.7041
Epoch 2/300
352/352 [==============================] - 1s 2ms/step - loss: 0.7815 - acc: 0.7197
Epoch 3/300
352/352 [==============================] - 1s 2ms/step - loss: 0.7551 - acc: 0.7272
Epoch 4/300
352/352 [==============================] - 1s 2ms/step - loss: 0.7395 - acc: 0.7325
Epoch 5/300
352/352 [==============================] - 1s 2ms/step - loss: 0.7269 - acc: 0.7371
Epoch 6/300
352/352 [==============================] - 1s 2ms/step - loss: 0.7167 - acc: 0.7393
Epoch 7/300
352/352 [==============================] - 1s 2ms/step - loss: 0.7073 - acc: 0.7446
Epoch 8/300
352/352 [==============================] - 1s 2ms/step - loss: 0.6985 - acc: 0.7470
Epoch 9/300
352/352 [==============================] - 1s 2ms/step - loss: 0.6898 - acc: 0.7524
Epoch 10/300
352/352 [==============================] - 1s 2ms/step - loss: 0.6828 - acc: 0.7555
Epoch 11/300
352/352 [=================

In [36]:
def plot_history(histories, key='categorical_crossentropy'):
  plt.figure(figsize=(16,10))

  for name, history in histories:
    val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

  plt.xlabel('Epochs')
  plt.ylabel(key.replace('_',' ').title())
  plt.legend()

  plt.xlim([0,max(history.epoch)])


plot_history([('baseline', baseline_history),
              ('l2', l2_model_history)])

KeyError: ignored

<Figure size 1152x720 with 0 Axes>

# 교차검증하기

In [ ]:
# kfold = KFold(n_splits=5, random_state=123, shuffle=True) # 기존 nsplit :5

In [ ]:
# result1 = cross_val_score(lgb_model,
#                          transform_X_train,
#                          y_train,
#                          cv = kfold
#                        )

# print(result1.mean())

In [ ]:
# print(lgb_model.score(transform_X_train,y_train))
# print(lgb_model.score(transform_X_test,y_test))